<a href="https://colab.research.google.com/github/13RENDA/MCTSinWatermark/blob/main/MCTS_in_WM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KGW repo

In [ ]:
!git clone https://github.com/jwkirchenbauer/lm-watermarking.git

Cloning into 'lm-watermarking'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 336 (delta 20), reused 10 (delta 6), pack-reused 289 (from 1)
Receiving objects: 100% (336/336), 12.00 MiB | 20.86 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [ ]:
%cd lm-watermarking

/content/lm-watermarking


In [ ]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130

In [ ]:
!python demo_watermark.py --gpt2 facebook/opt-6.7b

Traceback (most recent call last):
  File "/content/lm-watermarking/demo_watermark.py", line 24, in <module>
    import gradio as gr
ModuleNotFoundError: No module named 'gradio'


# KGW Demo

In [ ]:
# Example use of KGW watermark generator
# from extended_watermark_processor import WatermarkLogitsProcessor
from extended_watermark_processor import WatermarkLogitsProcessor
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList

# Assuming you are working with a pre-trained model like 'gpt2'
model_name = "gpt2"  # Replace with your desired model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name) # Load the model
new_tokens = 1
input_text = "The quick brown fox jumps over the"
# watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()))
watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
                                               gamma=0.25,
                                               delta=2.0,
                                               seeding_scheme="selfhash") #equivalent to `ff-anchored_minhash_prf-4-True-15485863`
# watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
#                                                gamma=0.25,
#                                                delta=2.0,
#                                                seeding_scheme="ff-anchored_minhash_prf-4-True-50256") #equivalent to `ff-anchored_minhash_prf-4-True-15485863`

# Note:
# You can turn off self-hashing by setting the seeding scheme to `minhash`.

tokenized_input = tokenizer(input_text, return_tensors='pt').to(model.device)
# note that if the model is on cuda, then the input is on cuda
# and thus the watermarking rng is cuda-based.
# This is a different generator than the cpu-based rng in pytorch!

output_tokens = model.generate(**tokenized_input,
                               logits_processor=LogitsProcessorList([watermark_processor]),
                               max_new_tokens = 25,
                               repetition_penalty=1.1)

# if decoder only model, then we need to isolate the
# newly generated tokens as only those are watermarked, the input/prompt is not
# output_tokens = output_tokens[:,tokenized_input["input_ids"].shape[-1]:]

output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
print(output_text)


ModuleNotFoundError: No module named 'extended_watermark_processor'

In [ ]:
output_tokens = model.generate(**tokenized_input,
                               logits_processor=LogitsProcessorList([watermark_processor]))
print(output_tokens)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[  464,  2068,  7586, 21831, 18045,   625,   262,  3355,    11,   475,
           339,  1595,   470,   760,   611,   339,   460,   466,   340,    13]])


In [ ]:
# Example use of KGW watermark detector
# from extended_watermark_processor import WatermarkDetector
from extended_watermark_processor import WatermarkDetector

# watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
#                                         gamma=0.5, # should match original setting
#                                         seeding_scheme="simple_1", # should match original setting
#                                         device=model.device, # must match the original rng device type
#                                         tokenizer=tokenizer,
#                                         z_threshold=4.0,
#                                         normalizers=[])

watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
                                        gamma=0.25, # should match original setting
                                        seeding_scheme="selfhash", # should match original setting
                                        device=model.device, # must match the original rng device type
                                        tokenizer=tokenizer,
                                        z_threshold=4.0,
                                        normalizers=[],
                                        ignore_repeated_ngrams=True)

# watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
#                                         gamma=0.25, # should match original setting
#                                         seeding_scheme="ff-anchored_minhash_prf-4-True-50256", # should match original setting
#                                         device=model.device, # must match the original rng device type
#                                         tokenizer=tokenizer,
#                                         z_threshold=4.0,
#                                         normalizers=[],
#                                         ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(output_text) # or any other text of interest to analyze
print(score_dict)

{'num_tokens_scored': 21, 'num_green_tokens': 7, 'green_fraction': 0.3333333333333333, 'z_score': 0.8819171036881969, 'p_value': 0.1889108185500319, 'z_score_at_T': tensor([-0.5774, -0.8165, -1.0000, -1.1547, -0.2582,  0.4714,  0.2182,  0.0000,
        -0.1925, -0.3651,  0.1741,  0.6667,  0.4804,  0.3086,  0.1491,  0.5774,
         0.4201,  0.8165,  0.6623,  1.0328,  0.8819]), 'prediction': False}


# MCTS in Watermark

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList
from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector  # Assuming this is the main class in your file
import math
import numpy as np
from collections import defaultdict

# Load model and tokenizer (you can replace 'gpt2' with a larger model if needed)
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


# Helper function for token entropy calculation
def calculate_entropy(logits):
    probabilities = F.softmax(logits, dim=-1)
    log_probabilities = F.log_softmax(logits, dim=-1)
    entropy = -(probabilities * log_probabilities).sum(-1)  # Summing over the vocabulary dimension
    return entropy

# Helper function for perplexity calculation
def compute_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs.input_ids

    # Calculate cross entropy loss
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss  # Cross entropy loss

    # Calculate perplexity
    perplexity = torch.exp(loss)
    return perplexity.item()

# Initialize the watermark processor
watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
                                               gamma=0.25,  # Adjust gamma if needed
                                               delta=2.0,  # Adjust delta if needed
                                               seeding_scheme="selfhash")  # Adjust seeding_scheme if needed

# Initialize the watermark detector

watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
                                       gamma=0.25,
                                       seeding_scheme="selfhash",
                                       device=model.device,
                                       tokenizer=tokenizer,
                                       z_threshold=4.0,
                                       normalizers=[],
                                       ignore_repeated_ngrams=True)

# Helper function for Monte Carlo Tree Search Node
class MCTSNode:
    def __init__(self, text, is_watermarked=False, parent=None):
        self.text = text
        self.is_watermarked = is_watermarked
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0

    def add_child(self, child_node):
        self.children.append(child_node)

# MCTS algorithm
class MCTS:
    def __init__(self, model, tokenizer, baseline_text, max_depth=10):
        self.model = model
        self.tokenizer = tokenizer
        self.baseline_text = baseline_text
        self.baseline_encoding = self.tokenizer.encode(baseline_text, return_tensors='pt')
        self.max_depth = max_depth

    def expand(self, node, current_depth):
        if current_depth >= self.max_depth:
            return node
        # Tokenize the current text to generate logits for the next possible tokens
        input_ids = self.tokenizer.encode(node.text, return_tensors='pt', padding=True)
        attention_mask = input_ids.new_ones(input_ids.shape)
        with torch.no_grad():
            outputs = self.model(input_ids, return_dict=True)
            logits = outputs.logits[:, -1, :]  # Get logits for the last generated token
            entropy = calculate_entropy(logits)

        # Sort tokens by entropy (descending order) and expand high-entropy tokens

        # k = min(5, entropy.shape[-1])
        # topk = torch.topk(entropy, k=k, dim=-1)
        # top_tokens = topk.indices.tolist()

        watermark_processor.rng = torch.Generator(device=input_ids.device)if watermark_processor.rng is None else watermark_processor.rng
        # Generate greenlist based on current input sequence
        greenlist_ids = watermark_processor._get_greenlist_ids(input_ids[0])

        # Generate the non-watermarked token (greedy choice)每一个子节点为前文+新词
        output_tokens_without_watermark = self.model.generate(input_ids, attention_mask = attention_mask, max_new_tokens=1)
        non_watermarked_token_id = output_tokens_without_watermark[0][-1].item()
        child_text_no_watermark = node.text + self.tokenizer.decode([non_watermarked_token_id], skip_special_tokens=True)
        print(f"leaf node w/o WM:{child_text_no_watermark}")
        # print(f"leaf node w/o WM:{self.tokenizer.decode([non_watermarked_token_id])}")
        child_node_no_watermark = MCTSNode(child_text_no_watermark, is_watermarked=False, parent=node)
        node.add_child(child_node_no_watermark)

        # Remove the non-watermarked token from the greenlist
        if non_watermarked_token_id in greenlist_ids:
            greenlist_ids = greenlist_ids[greenlist_ids != non_watermarked_token_id]


        # Expand into both watermarked and non-watermarked versions
        # green_list_ids = watermark_processor._get_greenlist_ids(input_ids, logits)
        for wm_token_id in greenlist_ids[:5]:
            # Generate the watermarked version
            # input_ids_with_watermark = self.tokenizer.encode(node.text + token, return_tensors='pt')
            # with torch.no_grad():
            #     output_tokens_with_watermark = self.model.generate(**{'input_ids': input_ids_with_watermark},
            #                                                          logits_processor=LogitsProcessorList([watermark_processor]),
            #                                                          max_new_tokens=1)
            child_text_with_watermark = node.text + self.tokenizer.decode([wm_token_id], skip_special_tokens=True)
            print(f"leaf node w/ WM:{child_text_with_watermark}")
            # print(f"leaf node w/o WM:{self.tokenizer.decode([wm_token_id])}")
            child_node_with_watermark = MCTSNode(child_text_with_watermark, is_watermarked=True,parent=node)
            node.add_child(child_node_with_watermark)

        return node

    def simulate(self, node):
        # Simulate until we reach the end or max depth用水印模型模拟
        tokenized_input = self.tokenizer(node.text, return_tensors="pt", padding=True).to(model.device)
        # attention_mask = tokenized_input["attention_mask"]

        with torch.no_grad():
            generated_outputs = self.model.generate(**tokenized_input,
                                                    logits_processor=LogitsProcessorList([watermark_processor]),
                                                    max_new_tokens=3,
                                                    repetition_penalty=1.1)

        generated_text = self.tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
        print(f"simulation result: {generated_text}")

        # Compute reward: combination of semantic similarity and watermark detectability
        reward = self.compute_reward(node, generated_text)
        return reward

    def compute_reward(self, node, generated_text):
        '''
        # Compute cosine similarity for semantic preservation

        baseline_encoding = self.tokenizer.encode(baseline_text[0], return_tensors='pt')
        generated_encoding = self.tokenizer.encode(generated_text[0], return_tensors='pt')
        cos_sim = F.cosine_similarity(baseline_encoding.float(), generated_encoding.float(), dim=-1)

        # Compute watermark detectability using WatermarkDetector
        score_dict = watermark_detector.detect(generated_text)
        detectability = score_dict['z_score']  # Or another relevant metric from score_dict

        # Reward is a combination of semantic similarity and watermark detectability
        alpha = 0  # Adjust the weight for detectability
        reward = cos_sim.item() + alpha * detectability
        '''
        perplexity = compute_perplexity(self.model, self.tokenizer, generated_text)
        reward = -perplexity
        if not node.is_watermarked:
            reward *= 8

        return reward

    def backpropagate(self, node, reward):
        # Backpropagate the reward to update the node's value
        # if node.parent != None:
        #   backpropagate(node.parent, reward)

        # node.visits += 1
        # if(node.value == float('-inf')):
        #   node.value = reward
        # else:
        #   node.value += reward
        # print(f"root: {node.text}; value:{node.value/node.visits if node.visits > 0 else float('-inf')}")
        if node.parent is not None:  # Check if node is not None before accessing parent
          self.backpropagate(node.parent, reward)
        node.visits += 1
        if(node.value == float('-inf')):
            node.value = reward
        else:
            node.value += reward
          # print(f"reward: {reward}")
        print(f"parent: {node.text};   {node.value/node.visits if node.visits > 0 else float('-inf')}")

    def search(self, root):
        for _ in range(50):  # Number of iterations
            node = root
            current_depth = 0
            print(f"root: {node.text};   {node.value/node.visits if node.visits > 0 else float('-inf')}")

            # Selection and Expansion
            while node.children and current_depth < self.max_depth:
                # UCB1 formula to select child
                node = max(node.children, key=lambda n: n.value / (n.visits + 1) + math.sqrt(2 * math.log(node.visits + 1) / (n.visits + 1)))
                current_depth += 1

            # Expansion
            if current_depth < self.max_depth:
              print("expansion:")
              node = self.expand(node, current_depth)

            # Simulation
            if node.parent is not None:
              reward = self.simulate(node)
              print(f"reward: {reward}")
              # Backpropagation
              print("backpropagate: ")
              self.backpropagate(node, reward)

        best_node = root
        while best_node.children:
          best_node = max(best_node.children, key=lambda n: n.value / n.visits if n.visits > 0 else float('inf'))
        return best_node.text


# Example usage:
prompt = "I'm so happy"
tokenized_input = tokenizer(prompt, return_tensors="pt", padding=True)
attention_mask = tokenized_input["attention_mask"]

# Generate a baseline response
baseline_response = model.generate(tokenizer.encode(prompt, return_tensors='pt'), attention_mask=attention_mask, max_new_tokens=8, repetition_penalty=1.1)
baseline_text = tokenizer.decode(baseline_response[0], skip_special_tokens=True)

# Initialize MCTS and perform search
root = MCTSNode(text=prompt)
mcts = MCTS(model, tokenizer, baseline_text, max_depth=5)
best_response = mcts.search(root)
print(f"baseline response: {baseline_text}")
print(f"Best response with watermark: {best_response}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


root: I'm so happy;   -inf
expansion:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


leaf node w/o WM:I'm so happy to
leaf node w/ WM:I'm so happy ambulance
leaf node w/ WM:I'm so happy meet
leaf node w/ WM:I'm so happy merger
leaf node w/ WM:I'm so happysych
leaf node w/ WM:I'm so happyenezuel
root: I'm so happy;   -inf
expansion:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

leaf node w/o WM:I'm so happy to be SH Died in
leaf node w/ WM:I'm so happy to be SH Diedorgetown
leaf node w/ WM:I'm so happy to be SH Died223
leaf node w/ WM:I'm so happy to be SH Died fossil
leaf node w/ WM:I'm so happy to be SH Died Og
leaf node w/ WM:I'm so happy to be SH Died Design


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


simulation result: I'm so happy to be SH Died in the womb
reward: -278.39971923828125
back
parent: I'm so happy;   -417.2932925934487
parent: I'm so happy to;   -322.48055439610636
parent: I'm so happy to be;   -202.06385221935454
parent: I'm so happy to be SH;   -190.0744400024414
parent: I'm so happy to be SH Died;   -278.39971923828125
root: I'm so happy;   -417.2932925934487
simulation result: I'm so happy to be hereVisit the website of a
reward: -531.3255615234375
back
parent: I'm so happy;   -419.6689648628235
parent: I'm so happy to;   -329.00696086883545
parent: I'm so happy to be;   -217.0302935513583
parent: I'm so happy to be here;   -188.61705356174045
parent: I'm so happy to be hereVisit;   -298.0542869567871
parent: I'm so happy to be hereVisit the;   -531.3255615234375
root: I'm so happy;   -419.6689648628235
expansion:


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


leaf node w/o WM:I'm so happy to Actually pandering
leaf node w/ WM:I'm so happy to Actually pand booze
leaf node w/ WM:I'm so happy to Actually pand Whale
leaf node w/ WM:I'm so happy to Actually pand laughed
leaf node w/ WM:I'm so happy to Actually pand Orders
leaf node w/ WM:I'm so happy to Actually pand knight
simulation result: I'm so happy to Actually pandering and not
reward: -322.27783203125
back
parent: I'm so happy;   -417.68139072340364
parent: I'm so happy to;   -328.8030478737571
parent: I'm so happy to Actually;   -311.20716094970703
parent: I'm so happy to Actually pand;   -322.27783203125
baseline response: I'm so happy to be here. I've been working
Best response with watermark: I'm so happy to be here deteriorating.
